In [ ]:
import glob, os, cv2, shutil
import matplotlib.pyplot as plt
import numpy as np
import moviepy.editor as mp
from PIL import Image

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2400256/45929032 bytes (5.2%)5898240/45929032 bytes (12.8%)9535488/45929032 bytes (20.8%)13197312/45929032 bytes (28.7%)16728064/45929032 bytes (36.4%)20496384/45929032 bytes (44.6%)24174592/45929032 bytes (52.6%)26738688/45929032 bytes (58.2%)30040064/45929032 bytes (65.4%)33669120/45929032 bytes (73.3%)37216256/45929032 bytes (81.0%)40943616/45929032 bytes (89.1%)

In [ ]:
depth_path = "PATH TO DEPTH VIDEOS"
seg_path = "PATH TO SEGMENTATION VIDEOS"
sal_path = "PATH TO SALIENCY VIDEOS"

comb_path = "PATH TO COMBINATION PATH"

## combine segmentation, saliency, on depth maps


In [ ]:
def seg_OR_saliency(video, threshold):
  fps = 20
  size = (960,540)
  vid_pathOut = comb_path + "/%s.avi" %video
  out = cv2.VideoWriter(vid_pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

  print(video)
  seg_v = cv2.VideoCapture('%s/%s.mp4' % (seg_path,video))
  sal_v = cv2.VideoCapture('%s/%s.mp4' % (sal_path,video))
  dep_v = cv2.VideoCapture('%s/%s.mp4' % (depth_path,video))

  success1 = 1
  success2 = 1
  success3 = 1

  while success1 and success2 and success3:
    success1, seg = seg_v.read()
    success2, sal = sal_v.read()
    success3, dep = dep_v.read()

    if success1 == 0 or success2 == 0 or success3 == 0:
      break

    sal_fil = sal.copy()
    threshold = np.max(sal_fil)*.90
    # print(threshold)
    sal_fil[sal_fil<=threshold] = 0
    sal_fil[sal_fil>0] = 255

    # normalize between [0,1]
    sal_norm = cv2.normalize(sal_fil, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    seg_norm = cv2.normalize(seg, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    dep_norm = cv2.normalize(dep, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
    seg_norm = cv2.resize(seg_norm, (sal_norm.shape[1],sal_norm.shape[0]))
    dep_norm = cv2.resize(dep_norm, (sal_norm.shape[1],sal_norm.shape[0]))
    seg_sal = np.max((sal_norm[:,:,0], seg_norm[:,:,0]), axis=0)
    dep_seg_sal = dep_norm[:,:,0].copy()
    dep_seg_sal[seg_sal==0] = 0

    result = dep_seg_sal.copy() * 255
    result3 = cv2.merge([result, result, result])
    out.write(np.uint8(result3))

  out.release()
  cv2.destroyAllWindows()